W problemie VertexCover mamy dany graf G = (V,E) oraz liczbę naturalną k. Należy sprawdzić, czy istnieje zbiór k wierzchołków takich, że każda krawędź dotyka co najmniej jednego wybranego wierzchołka. Jest to jeden z klasycznych problemów NP-zupełnych.

In [3]:
from dimacs import *
from itertools import combinations
from copy import deepcopy

G = loadGraph('./graph/e10')
E = edgeList(G)

## Brute-Force: O(n^k)

In [4]:
def vc_brute(G, E):
    for i in range(1, len(G)):
        for S in combinations(range(len(G)), i):
            if isVC(E, S):
                print(str(i) + ": " + str(S))
                return
            
vc_brute(G, E)

3: (3, 6, 10)


## Rekurencja z powrotami: O(2k)

Ten algorytm opiera się na obserwacji, że dla każdej krawędzi e = {u,v} należy wybrać co najmniej jeden z wierzchołków u i v. Algorytm działa więc następująco:

In [6]:
def VC_2k(E, k, S):
    # G to graf wejściowy, k liczba wierzchołków, które możemy użyć
    # S to zbiór wierzchołków, który budujemy

    # wybierz dowolną krawędź e = {u, v}, która nie jest
    # jeszcze pokryta(czyli ani u ani v nie jest wybrany)
    free_edge = None

    for i, e in enumerate(E):
        if e[0] not in S and e[1] not in S:
            free_edge = e
            edge_idx = i
            break

    if free_edge is None:
        return S  # rozwiązanie znalezione

    if k == 0:
        return None  # nie ma rozwiązania

    E = E[edge_idx:]

    u, v = free_edge

    return VC_k(E, k - 1, S.union({u})) or VC_k(E, k - 1, S.union({v}))


def vc_recursion_2k(G, E):
    for k in range(1, len(G)):
        S = set()
        S = VC_2k(E.copy(), k, S)
        if S:
            print(str(len(S)) + ": " + str(S))
            return
        
vc_recursion_2k(G, E)

3: {10, 3, 6}


## Rekurencja z powrotami: O(1.618k)

Ten algorytm opiera się na obserwacji, że jeśli rozważamy wierzchołek u, to do rozwiązania należy albo wziąć u, albo wszystkich jego sąsiadów (inaczej nie ma jak pokryć krawędzi łączących u z sąsiadami).

In [68]:
import copy

def solve(G, k, S):
    # G to graf wejściowy, k liczba wierzchołków, które możemy użyć
    # S to zbiór wierzchołków, który budujemy
    E = edgeList(G)
    if k <= 0:
        return None
    if len(E) == 0:
        return S

    u, v = E[0]
    new_set_1 = S.copy()
    new_set_1.add(u)
    new_graph = copy.deepcopy(G)
    for vertex_connections in new_graph:
        if u in vertex_connections:
            vertex_connections.remove(u)
    new_graph[u] = set()
    S1 = solve(new_graph, k - 1, new_set_1)
    if S1:
        return S1
    
    new_graph_2 = copy.deepcopy(G)
    current_neightbour = new_graph_2[u].copy()
    for neighbour in current_neightbour:
        for vertex_connections in new_graph_2:
            if neighbour in vertex_connections:
                vertex_connections.remove(neighbour)
        new_graph_2[neighbour] = set()
    S2 = solve(new_graph_2 ,k - len(G[u]) ,set.union(S.copy(), G[u]))

    return S2
    
def vs_recursion_1618k(G):
    for k in range(1, len(G)):
        S = set()
        S = solve(G, k, S)
        if S and isVC(E, S):
            print(str(len(S)) + ": " + str(S))
            return
            
G = loadGraph('./graph/e10')
vs_recursion_1618k(G)

3: {10, 3, 6}


## Rekurencja z powrotami: O(1.47k)

To ten sam algorytm, co powyżej ale z rozsądniejszym wyborem wierzchołków u. Za każdym razem wybieramy wierzchołek o jak największym stopniu (a przynajmniej o stopniu większym od 2). Gdy zostaną same wierzchołki o stopniu 1 i 2, to rozwiązujemy problem w czasie wielomianowym (wówczas graf to zbiór ścieżek i cykli i powinni Państwo być w stanie wskazać algorytm wielomianowy samodzielnie).

Alternatywne podejście (łatwiejsze w implementacji):

* gdy mamy choć jeden wierzchołek o stopniu 1 to wiadomo, że należy do rozwiązania dodać jego jedynego sąsiada (i usunąć z grafu u i N(u))
* jeśli nie ma żadnego wierzchołka o stopniu 1, to wybieramy wierzchołek o jak najwyższym stopniu (zwróćmy uwagę, że jeśli zostały nam same wierzchołki o stopniu 2, to tworzą one cykle i każdy cykl rozetniemy w jednym zejściu rekurencyjnym tak, że pojawią się wierzchołki o stopniu 1)